In [ ]:
import numpy as np
import pandas as pd
import DTA.polarDensity_helper as pc
from DTA.enrichment_plotters import polar_plot
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
plt.rcParams['axes.grid'] = False 

# Important User Settings:

In [ ]:
root = Path("/Users/ezry/Projects/ELIC_PE_PG/liam/trajectory_version")
chains_groups = ["POPE", "POPG"]
lipids = chains_groups

In [ ]:
enrich = True

# get files to use

file_list = []
leaflets = ['low', 'upp']
for lip in lipids:
    for leaf in leaflets:
        toadd = list(root.glob(f"{lip}.dat.{leaf}.avg.dat") )
        file_list = np.append(file_list,toadd)
assert len(file_list) > 0, "Error, you don't appear to have files that match that path and regex."

In [ ]:
enrichments = pd.DataFrame(index=chains_groups, columns=leaflets)
counts = pd.DataFrame(index=chains_groups, columns=leaflets)
myenrichments = pd.DataFrame(index=chains_groups, columns=leaflets)

idx = 0
for fl in file_list:
    if idx == 0:
        rad, dr, dth, theta, radius, frames, Ntheta = pc.Coord_Get(fl)
        A = (radius * dr * dth)
        thetas = np.unique(theta)

    filename = fl.name

    tmp_chain = filename.split('.')[0]
    tmp_nm = filename.split('.')[2]

    # This is a hack. The above part does not have a "flexible"
    # method to consider sim type (a, b ...)
    idx+=1
    toadd = np.loadtxt(fl, skiprows=1)
    toadd = toadd[:,3:-1]
    data, num_mol,avg_A,beads,exrho,avg_chain = pc.get_polar_data(fl)
    counts.at[tmp_chain,tmp_nm] = toadd
    myenrichments.at[tmp_chain,tmp_nm] = (toadd/A)/exrho
    enrichments.at[tmp_chain,tmp_nm] = pc._analysis_call_(fl, radius, dr, dth, frames, enrich=enrich)


In [ ]:
# Optional helix locations
try:
    helices_lwr = np.loadtxt(root.joinpath("Protein_coords_lwr.dat"))
    helices_upr = np.loadtxt(root.joinpath("Protein_coords_upr.dat"))
except FileNotFoundError:
    helices_lwr = None
    helices_upr = None
    print("Protein coordinates not found")

In [ ]:
bins = {1:(range(4,30,6), np.repeat(27.5,5)), 
        2:(range(5,30,6), np.repeat(27.5,5)),
        3:(range(4,30,6), np.repeat(32.5,5)),
        4:(range(5,30,6), np.repeat(32.5,5)),
        5:(range(4,30,6), np.repeat(22.5,5))}
# axes[i].scatter(thetas[2], np.repeat(27.5,1), color=color, marker="x", linewidth=7)
# axes[i].scatter(thetas[bins[1][0]], bins[1][1], color=color, marker="x", linewidth=7)
# #axes[i].scatter(thetas[4:30:6], np.repeat(32.5,5), color=color, marker="x", linewidth=7)
# #axes[i].scatter(thetas[5:30:6], np.repeat(32.5,5), color=color, marker="x", linewidth=7)
# #axes[i].scatter(thetas[4:30:6], np.repeat(22.5,5), color=color, marker="x", linewidth=7)

In [ ]:
fig, axes = polar_plot(enrichments, theta, radius, chains_groups, helices_lwr=helices_lwr, helices_upr=helices_upr, colorbychain=True, vmin=0, vmax=2, vmid=1)
for i in range(len(axes)):
    if i == 2:
        color = "lime"
    else:
        color = "purple"
    for bin in bins.values():
        axes[i].scatter(thetas[bin[0]], bin[1], color=color, marker="x", linewidth=7)

plt.savefig(root.joinpath("enrichments.pdf"))
plt.show()